# Homework 3

Due Thursday May 06, 2021

# Classification with one hidden layer Neural Network

In this assignment you will build your first neural network, which will have a single hidden layer. You will see a big difference between this model and the one we implemented using logistic regression. 

**You will perform the following tasks:**
- Implement a 2-class classification neural network with a single hidden layer
- Use units with a non-linear activation function, such as tanh 
- Compute the cross entropy loss 
- Implement forward and backward propagation

# Dataset and Helper Code

- You may download the smaller set of cats and dogs [dataset](https://github.com/nayeemmz/cmsc426Spring2021/blob/master/Data/catsanddogs.zip) for the assignment from the Github page.
- You may find the [Logistic Regression](https://github.com/nayeemmz/cmsc426Spring2021/blob/master/Notebooks/LogRegressionUsingNeuralNetworks.ipynb) notebook shown in class useful for this homework.

## 1 - Packages ##

Import all the packages that you will need during this assignment.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from skimage.transform import resize
import os, shutil
%matplotlib inline

## 2 - Overview of the Problem set ##

**Problem Statement**: We have a dataset containing:
    - a training set of m_train(2000) images labeled as cat (y=1) or dog (y=0)
    - a test set of m_test(1000) images labeled as cat or dog
    - each image is of different shape. We reshape it to be (num_px, num_px, 3) where 3 is for the 3 channels (RGB). Thus, each image is square (height = num_px) and (width = num_px).

We will build a simple image-recognition algorithm that can correctly classify pictures as cat or dog.

Let's get more familiar with the dataset. Load the data.

In [ ]:
# Load your cats/dogs data set here as train_set_x_orig, test_set_x_orig, 
# and their labels as train_set_y, test_set_y

### START CODE HERE ### 




### END CODE HERE ###


In [ ]:
print(train_set_x_orig.shape)
print(test_set_x_orig.shape)
print(train_set_y.shape)
print(test_set_y.shape)

(These outputs may vary depending on the number of images you use)

**Expected Output for train_set_x_orig, test_set_x_orig, train_set_y and test_set_y**: 
<table style="width:25%">
  <tr>
    <td>**train_set_x_orig**</td>
    <td> (2000, 200, 200, 3) </td> 
  </tr>
  
  <tr>
    <td>**test_set_x_orig**</td>
    <td> (1000, 200, 200, 3) </td> 
  </tr>
  
  <tr>
    <td>**train_set_y**</td>
    <td> (1, 2000) </td> 
  </tr>
    
  <tr>
    <td>**test_set_y**</td>
    <td> (1, 1000) </td> 
  </tr>
  
</table>

We added "_orig" at the end of image datasets (train and test) because we are going to preprocess them. After preprocessing, we will end up with train_set_x and test_set_x (the labels train_set_y and test_set_y don't need any preprocessing).

Each line of your train_set_x_orig and test_set_x_orig is an array representing an image. You can visualize an example by running the following code. Feel free also to change the `index` value and re-run to see other images.

In [ ]:
# Show an Example of a picture
index = 25
plt.imshow(train_set_x_orig[index])
print ("y = " + str(train_set_y[:, index]))


Many software bugs in deep learning come from having matrix/vector dimensions that don't fit. If you can keep your matrix/vector dimensions straight you will go a long way toward eliminating many bugs. 

**Exercise:** Find the values for:
    - m_train (number of training examples)
    - m_test (number of test examples)
    - num_px (= height = width of a training image)
Remember that `train_set_x_orig` is a numpy-array of shape (m_train, num_px, num_px, 3). For instance, you can access `m_train` by writing `train_set_x_orig.shape[0]`.

In [ ]:
### START CODE HERE ### 


### END CODE HERE ###

print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Height/Width of each image: num_px = " + str(num_px))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_set_x shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x shape: " + str(test_set_x_orig.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

**Expected Output for m_train, m_test and num_px**: 
<table style="width:15%">
  <tr>
    <td>**m_train**</td>
    <td> 2000 </td> 
  </tr>
  
  <tr>
    <td>**m_test**</td>
    <td> 1000 </td> 
  </tr>
  
  <tr>
    <td>**num_px**</td>
    <td> 200 </td> 
  </tr>
  
</table>

For convenience, you should now reshape images of shape (num_px, num_px, 3) in a numpy-array of shape (num_px $*$ num_px $*$ 3, 1). After this, our training (and test) dataset is a numpy-array where each column represents a flattened image. There should be m_train (respectively m_test) columns.

**Exercise:** Reshape the training and test data sets so that images of size (num_px, num_px, 3) are flattened into single vectors of shape (num\_px $*$ num\_px $*$ 3, 1).

A trick when you want to flatten a matrix X of shape (a,b,c,d) to a matrix X_flatten of shape (b$*$c$*$d, a) is to use: 
```python
X_flatten = X.reshape(X.shape[0], -1).T      # X.T is the transpose of X

In [ ]:
# Reshape the training and test examples

### START CODE HERE ###


### END CODE HERE ###

print ("train_set_x_flatten shape: " + str(train_set_x_flatten.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x_flatten shape: " + str(test_set_x_flatten.shape))
print ("test_set_y shape: " + str(test_set_y.shape))
print ("sanity check after reshaping: " + str(train_set_x_flatten[0:5,0]))

**Expected Output**: 

<table style="width:75%">
  <tr>
    <td>**train_set_x_flatten shape**</td>
    <td> (120000, 2000)</td> 
  </tr>
  <tr>
    <td>**train_set_y shape**</td>
    <td>(1, 2000)</td> 
  </tr>
  <tr>
    <td>**test_set_x_flatten shape**</td>
    <td>(120000, 1000)</td> 
  </tr>
  <tr>
    <td>**test_set_y shape**</td>
    <td>(1, 1000)</td> 
  </tr>
  <tr>
  <td>**sanity check after reshaping**</td>
  <td>[0.37758824 0.22596078 0.17988235 0.36972549 0.22745098]</td> 
  </tr>
</table>

To represent color images, the red, green and blue channels (RGB) must be specified for each pixel, and so the pixel value is actually a vector of three numbers ranging from 0 to 255.
One common preprocessing step in machine learning is to center and standardize your dataset, meaning that you substract the mean of the whole numpy array from each example, and then divide each example by the standard deviation of the whole numpy array. But for picture datasets, it is simpler and more convenient and works almost as well to just divide every row of the dataset by 255 (the maximum value of a pixel channel).
Let's standardize our dataset.

In [1]:
# Standardize the dataset

### START CODE HERE ###


### END CODE HERE ###

<font color='blue'>
**What you need to remember:**

Common steps for pre-processing a new dataset are:
- Figure out the dimensions and shapes of the problem (m_train, m_test, num_px, ...)
- Reshape the datasets such that each example is now a vector of size (num_px \* num_px \* 3, 1)
- "Standardize" the data

## 3 - Neural Network model

Logistic regression did not work well on the "cats/dogs dataset". You are going to train a Neural Network with a single hidden layer.

**Here is our model**:
<img src="images/OneLayerNN.png" >

**Mathematically**:

For one example $x_{i}$:
$$z_{1 i} =  W_{1} x_{i} + b_{1 i}\tag{1}$$ 

$$a_{1 i} = \tanh(z_{1 i})\tag{2}$$

$$z_{2 i} = W_{2} a_{1 i} + b_{2 i}\tag{3}$$

$$\hat{y}_{i} = a_{2 i} = \sigma(z_{ 2 i})\tag{4}$$

$$\hat{y}_{i} = \begin{cases} 1 & \mbox{if } a_{2i} > 0.5 \\ 0 & \mbox{otherwise } \end{cases}\tag{5}$$

Given the predictions on all the examples, you can also compute the cost $J$ as follows: 


<img src="images/lossfunc.png" style="width:175px;height:18px;"> $$\tag{6}$$



**Reminder**: The general methodology to build a Neural Network is to:
    1. Define the neural network structure ( # of input units,  # of hidden units, etc). 
    2. Initialize the model's parameters
    3. Loop:
        - Implement forward propagation
        - Compute loss
        - Implement backward propagation to get the gradients
        - Update parameters (gradient descent)

You often build helper functions to compute steps 1-3 and then merge them into one function we call `nn_model()`. Once you've built `nn_model()` and learnt the right parameters, you can make predictions on new data.

### 3.1 - Defining the neural network structure ####

**Exercise**: Define three variables:
    - n_x: the size of the input layer
    - n_h: the size of the hidden layer (set this to 4) 
    - n_y: the size of the output layer

**Hint**: Use shapes of X and Y to find n_x and n_y. Also, hard code the hidden layer size to be 4. You can change it later if you want to try different values.

In [2]:
def layer_sizes(X, Y):
    """
    Arguments:
    X -- input dataset of shape (input size, number of examples)
    Y -- labels of shape (output size, number of examples)
    
    Returns:
    n_x -- the size of the input layer
    n_h -- the size of the hidden layer
    n_y -- the size of the output layer
    """
    ### START CODE HERE ### 
           # size of input layer
    
           # size of output layer
    ### END CODE HERE ###
    return (n_x, n_h, n_y)

In [ ]:
(n_x, n_h, n_y) = layer_sizes(train_set_x_flatten, train_set_y)
print("The size of the input layer is: n_x = " + str(n_x))
print("The size of the hidden layer is: n_h = " + str(n_h))
print("The size of the output layer is: n_y = " + str(n_y))

 **Expected Output**: 



<table style="width:20%">
  <tr>
    <td>**n_x**</td>
    <td> 120000 </td> 
  </tr>
  <tr>
    <td>**n_h**</td>
    <td>4</td> 
  </tr>
  <tr>
    <td>**n_y**</td>
    <td>1</td> 
  </tr>
  
  
</table>

### 3.2 - Initialize the model's parameters ####

**Exercise**: Implement the function `initialize_parameters()`.

**Instructions**:
- Make sure your parameters' sizes are right. Refer to the neural network figure above if needed.
- You will initialize the weights matrices with random values. 
    - Use: `np.random.randn(a,b) * 0.01` to randomly initialize a matrix of shape (a,b).
- You will initialize the bias vectors as zeros. 
    - Use: `np.zeros((a,b))` to initialize a matrix of shape (a,b) with zeros.

In [ ]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed(2) # we set up a seed so that your output matches ours although the initialization is random.
    
    ### START CODE HERE ### (≈ 4 lines of code)
    
    ### END CODE HERE ###
    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [ ]:
parameters = initialize_parameters(n_x, n_h, n_y)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Expected Output**:

<table style="width:50%">
  <tr>
    <td>**W1**</td>
    <td> [[-0.00416758 -0.00056267 -0.02136196 ... -0.02239253  0.02158785  -0.00359388]
 [ 0.01477743 -0.00170422 -0.01070759 ...  0.01287999 -0.01210301
  -0.00707942]
 [ 0.0167966  -0.00593004 -0.0012511  ... -0.00292221 -0.00433734
   0.001981  ]
 [ 0.00947413 -0.02232344 -0.00625281 ...  0.01042966  0.00975072
   0.02265885]] </td> 
  </tr>
  
  <tr>
    <td>**b1**</td>
    <td> [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]] </td> 
  </tr>
  
  <tr>
    <td>**W2**</td>
    <td> [[ 0.00043097 -0.01053257 -0.00601822 -0.0041587 ]]</td> 
  </tr>
  

  <tr>
    <td>**b2**</td>
    <td> [[ 0.]] </td> 
  </tr>
  
</table>

### 3.3 - The Loop ####

**Question**: Implement `forward_propagation()`.

**Instructions**:
- Look above at the mathematical representation of your classifier.
- You can use the function `sigmoid()`. It is included in the notebook.
- You can use the function `np.tanh()`. It is part of the numpy library.
- The steps you have to implement are:
    1. Retrieve each parameter from the dictionary "parameters" (which is the output of `initialize_parameters()`) by using `parameters[".."]`.
    2. Implement Forward Propagation. Compute $Z_{1}, A_{1}, Z_{2}$ and $A_{2}$ (the vector of all your predictions on all the examples in the training set).
- Values needed in the backpropagation are stored in "`cache`". The `cache` will be given as an input to the backpropagation function.

In [ ]:
def sigmoid(z):
    """
    Compute the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(z)
    """

 
    s = 1/(1+np.exp(-z))
   
    
    return s

def forward_propagation(X, parameters):
    """
    Argument:
    X -- input data of size (n_x, m)
    parameters -- python dictionary containing your parameters (output of initialization function)
    
    Returns:
    A2 -- The sigmoid output of the second activation
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"
    """
    # Retrieve each parameter from the dictionary "parameters"
    ### START CODE HERE ### (≈ 4 lines of code)
    
    ### END CODE HERE ###
    
    # Implement Forward Propagation to calculate A2 (probabilities)
    ### START CODE HERE ### (≈ 4 lines of code)
    
    ### END CODE HERE ###
    
    assert(A2.shape == (1, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [ ]:
A2, cache = forward_propagation(train_set_x_flatten, parameters)

# Note: we use the mean here just to make sure that your output matches ours. 
print(np.mean(cache['Z1']) ,np.mean(cache['A1']),np.mean(cache['Z2']),np.mean(cache['A2']))

**Expected Output**:
<table style="width:50%">
  <tr>
    <td> -1.2567265585333875 -0.6404675075964693 0.009915869468064713 0.5024789240379566 </td> 
  </tr>
</table>

Now that you have computed $A_{2}$ (in the Python variable "`A2`"), which contains $a_{2i}$ for every example, you can compute the cost function as follows:

$$J = - \frac{1}{m} \sum\limits_{i = 0}^{m} \large{(} \small y_{i}\log\left(a_{2i}\right) + (1-y_{i})\log\left(1- a_{2 i}\right) \large{)} \small\tag{13}$$

**Exercise**: Implement `compute_cost()` to compute the value of the cost $J$.

**Instructions**:
- There are many ways to implement the cross-entropy loss. To help you, we give you how we would have implemented
$- \sum\limits_{i=0}^{m}  y^{(i)}\log(a^{[2](i)})$:
```python
logprobs = np.multiply(np.log(A2),Y)
cost = - np.sum(logprobs)                # no need to use a for loop!
```

(you can use either `np.multiply()` and then `np.sum()` or directly `np.dot()`).

In [ ]:
def compute_cost(A2, Y, parameters):
    """
    Computes the cross-entropy cost given in equation (13)
    
    Arguments:
    A2 -- The sigmoid output of the second activation, of shape (1, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    parameters -- python dictionary containing your parameters W1, b1, W2 and b2
    
    Returns:
    cost -- cross-entropy cost given equation (13)
    """
    
    m = Y.shape[1] # number of example

    # Compute the cross-entropy cost
    ### START CODE HERE ### (≈ 2 lines of code)
    
    ### END CODE HERE ###
    
    cost = np.squeeze(cost)     # makes sure cost is the dimension we expect. 
                                # E.g., turns [[17]] into 17 
    assert(isinstance(cost, float))
    
    return cost

In [ ]:
print("cost = " + str(compute_cost(A2, train_set_y, parameters)))

**Expected Output**:
<table style="width:20%">
  <tr>
    <td>**cost**</td>
    <td> 0.692923... </td> 
  </tr>
  
</table>

Using the cache computed during forward propagation, you can now implement backward propagation.

**Question**: Implement the function `backward_propagation()`.

**Instructions**:
Backpropagation is usually the hardest (most mathematical) part in deep learning. To help you, here again is the slide from the lecture on backpropagation. You'll want to use the six equations on the right of this slide, since you are building a vectorized implementation.  

<img src="images/gradDesc.png" >

<!--
$\frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } = \frac{1}{m} (a^{[2](i)} - y^{(i)})$

$\frac{\partial \mathcal{J} }{ \partial W_2 } = \frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } a^{[1] (i) T} $

$\frac{\partial \mathcal{J} }{ \partial b_2 } = \sum_i{\frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)}}}$

$\frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)} } =  W_2^T \frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } * ( 1 - a^{[1] (i) 2}) $

$\frac{\partial \mathcal{J} }{ \partial W_1 } = \frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)} }  X^T $

$\frac{\partial \mathcal{J} _i }{ \partial b_1 } = \sum_i{\frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)}}}$

- Note that $*$ denotes elementwise multiplication.
- The notation you will use is common in deep learning coding:
    - dW1 = $\frac{\partial \mathcal{J} }{ \partial W_1 }$
    - db1 = $\frac{\partial \mathcal{J} }{ \partial b_1 }$
    - dW2 = $\frac{\partial \mathcal{J} }{ \partial W_2 }$
    - db2 = $\frac{\partial \mathcal{J} }{ \partial b_2 }$
    
!-->

- Tips:
    - To compute dZ1 you'll need to compute the gradient of tanh activation function, if $a_1 = tanh(z)$ then $\frac{\partial a_1}{\partial z} = 1-a_1^2$. So you can compute 
    using `(1 - np.power(A1, 2))`.

In [3]:
def backward_propagation(parameters, cache, X, Y):
    """
    Implement the backward propagation using the instructions above.
    
    Arguments:
    parameters -- python dictionary containing our parameters 
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2".
    X -- input data of shape (2, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    m = X.shape[1]
    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    ### START CODE HERE ### (≈ 2 lines of code)
    
    ### END CODE HERE ###
        
    # Retrieve also A1 and A2 from dictionary "cache".
    ### START CODE HERE ### (≈ 2 lines of code)
    
    ### END CODE HERE ###
    
    # Backward propagation: calculate dW1, db1, dW2, db2. 
    ### START CODE HERE ### (≈ 6 lines of code, corresponding to 6 equations on slide above)
    
    ### END CODE HERE ###
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [ ]:
grads = backward_propagation(parameters, cache, train_set_x_flatten, train_set_y)
print ("dW1 = "+ str(grads["dW1"]))
print ("db1 = "+ str(grads["db1"]))
print ("dW2 = "+ str(grads["dW2"]))
print ("db2 = "+ str(grads["db2"]))

**Expected output**:



<table style="width:80%">
  <tr>
    <td>**dW1**</td>
    <td> [[-1.91633059e-07  5.10321011e-07  8.97563180e-08 ... -1.68521075e-06
  -1.28012640e-06 -2.15469204e-06]
 [-7.74615735e-05 -1.18746330e-04 -9.29136819e-05 ... -9.61327786e-05
  -1.20296728e-04 -7.14529188e-05]
 [-1.23537532e-05 -2.10401754e-05 -1.56479254e-05 ... -1.08156990e-05
  -1.43071347e-05 -3.93107880e-06]
 [-7.24706488e-06 -1.30817750e-05 -8.80013989e-06 ... -1.11804236e-05
  -1.31926059e-05 -6.84879954e-06]] </td> 
  </tr>
  
  <tr>
    <td>**db1**</td>
    <td>  [[-9.77637081e-07]
 [-1.62816582e-04]
 [-1.94490806e-05]
 [-7.23672186e-06]] </td> 
  </tr>
  
  <tr>
    <td>**dW2**</td>
    <td> [[-7.11527991e-03  1.97213038e-02 -8.46238347e-06 -9.65836091e-04]] </td> 
  </tr>
  

  <tr>
    <td>**db2**</td>
    <td> [[0.00247892]] </td> 
  </tr>
  
</table> 

**Question**: Implement the update rule. Use gradient descent. You have to use (dW1, db1, dW2, db2) in order to update (W1, b1, W2, b2).

**General gradient descent rule**: $ \theta = \theta - \alpha \frac{\partial J }{ \partial \theta }$ where $\alpha$ is the learning rate and $\theta$ represents a parameter.

In [4]:
def update_parameters(parameters, grads, learning_rate = 1.2):
    """
    Updates parameters using the gradient descent update rule given above
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients 
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    # Retrieve each parameter from the dictionary "parameters"
    ### START CODE HERE ### (≈ 4 lines of code)
    
    ### END CODE HERE ###
    
    # Retrieve each gradient from the dictionary "grads"
    ### START CODE HERE ### (≈ 4 lines of code)
    
    ## END CODE HERE ###
    
    # Update rule for each parameter
    ### START CODE HERE ### (≈ 4 lines of code)
    
    ### END CODE HERE ###
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [ ]:
parameters = update_parameters(parameters, grads)

print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Expected Output**:


<table style="width:80%">
  <tr>
    <td>**W1**</td>
    <td> [[-0.00416735 -0.00056328 -0.02136207 ... -0.02239051  0.02158939
  -0.00359129]
 [ 0.01487039 -0.00156172 -0.01059609 ...  0.01299535 -0.01195865
  -0.00699368]
 [ 0.01681143 -0.00590479 -0.00123232 ... -0.00290923 -0.00432017
   0.00198572]
 [ 0.00948283 -0.02230775 -0.00624225 ...  0.01044307  0.00976655
   0.02266707]]</td> 
  </tr>
  
  <tr>
    <td>**b1**</td>
    <td> [[1.17316450e-06]
 [1.95379899e-04]
 [2.33388967e-05]
 [8.68406623e-06]]</td> 
  </tr>
  
  <tr>
    <td>**W2**</td>
    <td> [[ 0.0089693  -0.03419813 -0.00600807 -0.00299969]] </td> 
  </tr>
  

  <tr>
    <td>**b2**</td>
    <td> [[-0.00297471]] </td> 
  </tr>
  
</table>  

### 3.4 - Integrate parts 4.1, 4.2 and 4.3 in nn_model() ####

**Question**: Build your neural network model in `nn_model()`.

**Instructions**: The neural network model has to use the previous functions in the right order.

In [ ]:
def nn_model(X, Y, n_h, num_iterations = 100000, print_cost=False):
    """
    Arguments:
    X -- dataset of shape (2, number of examples)
    Y -- labels of shape (1, number of examples)
    n_h -- size of the hidden layer
    num_iterations -- Number of iterations in gradient descent loop
    print_cost -- if True, print the cost every 1000 iterations
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    # Initialize parameters, then retrieve W1, b1, W2, b2. Inputs: "n_x, n_h, n_y". Outputs = "W1, b1, W2, b2, parameters".
    ### START CODE HERE ### (≈ 5 lines of code)
    
    ### END CODE HERE ###
    
    # Loop (gradient descent)

    for i in range(0, num_iterations+1):
         
        ### START CODE HERE ### (≈ 4 lines of code)
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        
        
        # Cost function. Inputs: "A2, Y, parameters". Outputs: "cost".
        
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        
 
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        
        
        ### END CODE HERE ###
        
        # Print the cost every 1000 iterations
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters

In [ ]:
parameters = nn_model(train_set_x_flatten, train_set_y, 4, num_iterations=10000, print_cost=True)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

### 3.5 Predictions

**Question**: Use your model to predict by building predict().
Use forward propagation to predict results.

**Reminder**: predictions = $y_{prediction} = \mathbb 1 \text{{activation > 0.5}} = \begin{cases}
      1 & \text{if}\ activation > 0.5 \\
      0 & \text{otherwise}
    \end{cases}$  
    
As an example, if you would like to set the entries of a matrix X to 0 and 1 based on a threshold you would do: ```X_new = (X > threshold)```

In [ ]:
def predict(parameters, X):
    """
    Using the learned parameters, predicts a class for each example in X
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data of size (n_x, m)
    
    Returns
    predictions -- vector of predictions of our model (red: 0 / blue: 1)
    """
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    ### START CODE HERE ### (≈ 2 lines of code)
    
    ### END CODE HERE ###
    
    return predictions

In [ ]:
predictions = predict(parameters, train_set_x_flatten)
print("predictions mean = " + str(np.mean(predictions)))

In [ ]:
# Build a model with a n_h-dimensional hidden layer
parameters = nn_model(train_set_x_flatten, train_set_y, n_h = 4, num_iterations = 10000, print_cost=True)

In [ ]:
# Print accuracy
predictions = predict(parameters, train_set_x_flatten)
print ('Accuracy: %d' % float((np.dot(train_set_y,predictions.T) + np.dot(1-train_set_y,1-predictions.T))/float(train_set_y.size)*100) + '%')

### Start code here for test accuracy ###


### END CODE HERE ###

### 3.6 - Tuning hidden layer size  ###

In the cell below, write code to observe different behaviors of the model for various hidden layer sizes.

In [ ]:
# Write code to test different hidden layer size and its impact on accuracy
# You will require a for-loop for the various hidden layer sizes

### START CODE### 


### END CODE ###

## 4 - Report

In the cell below write markdown code to analyze the following:


- Write the observations about the ability of the larger models (with more hidden units) to fit the training data better. Eventually does it overfit the data (accuracy on the test set drops or not)? 
- Write about the best hidden layer size.
- Train for different number of iterations.
- Try few different values of the learning rate and report its effect.